# Telegram API Access
Detects suspicious requests to Telegram API without the usual Telegram User-Agent

## Rule Content
```
- title: Telegram API Access
  id: b494b165-6634-483d-8c47-2026a6c52372
  status: experimental
  description: Detects suspicious requests to Telegram API without the usual Telegram
    User-Agent
  references:
  - https://researchcenter.paloaltonetworks.com/2018/03/unit42-telerat-another-android-trojan-leveraging-telegrams-bot-api-to-target-iranian-users/
  - https://blog.malwarebytes.com/threat-analysis/2016/11/telecrypt-the-ransomware-abusing-telegram-api-defeated/
  - https://www.welivesecurity.com/2016/12/13/rise-telebots-analyzing-disruptive-killdisk-attacks/
  author: Florian Roth
  date: 2018/06/05
  logsource:
    category: proxy
    product: null
    service: null
  detection:
    selection:
      r-dns:
      - api.telegram.org
    filter:
      c-useragent:
      - '*Telegram*'
      - '*Bot*'
    condition: selection and not filter
  fields:
  - ClientIP
  - c-uri
  - c-useragent
  falsepositives:
  - Legitimate use of Telegram bots in the company
  level: medium

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(r-dns:("api.telegram.org") AND (NOT (c-useragent.keyword:(*Telegram* OR *Bot*))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()